In [1]:
from bs4 import BeautifulSoup
import requests
import splinter
import selenium

In [3]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204%3A19&blank_scope=Latest"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')

In [13]:
#print(soup.prettify())

In [5]:
container_box = soup.find_all('div', class_='image_and_description_container')

In [12]:
#link = soup.find_all('li', class_='slide')
title = container_box[0].find_all('img')[1]['alt']
para = container_box[0].find_all('a')[0].text.strip()

In [14]:
title

'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing'

In [15]:
para

"Learn more about the agency's next Red Planet mission during a live event on June 17."

In [16]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
jpl_request = requests.get(jpl_url)
jpl_soup = BeautifulSoup(jpl_request.text, 'html.parser')

In [118]:
jpl_soup

<!DOCTYPE html>

<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html>
<!-- START HEADER: "DEFAULT" -->
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push(
{'gtm.start': new Date().getTime(),event:'gtm.js'}
);var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-NLDQZ25');</script>
<!-- End Google Tag Manager -->
<head>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/> <title>Space Images</title>
<link href="/assets/stylesheets/manifest.css" media="all" rel="stylesheet" type="text

In [17]:
#new_var = jpl_soup.find_all('div', class_='default floating_text_area ms-layer')
new_var = jpl_soup.find_all('footer')
new_var[0]

<footer>
<a class="button fancybox" data-description="This map shows the route driven by NASA's Mars rover Curiosity through the 43rd Martian day, or sol, of the rover's mission on Mars (Sept. 19, 2012). The route starts where the rover touched down, a site subsequently named Bradbury Landing." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA16153_ip.jpg" data-link="/spaceimages/details.php?id=PIA16153" data-title="Curiosity Traverse Map Through Sol 43" id="full_image">
					FULL IMAGE
				  </a>
</footer>

In [131]:
new_var_2 = new_var[0]['data-link']
new_var_2

KeyError: 'data-link'

In [ ]:
new_var_2 = new_var[0]['data-link']